In [2]:
from keras.layers import Input, Conv2D, Activation, BatchNormalization
from keras.layers.merge import Add
from keras.layers.core import Dropout

In [ ]:
def gen(self):
    model = Sequential()
    #input layer
    model.add(Dense(20, input_shape = (10,)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    #layer 2
    model.add(Dense(40))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    #layer 3
    model.add(Dense(80))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(10, activation='tanh'))
    
    return model